In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys
import pymedphys._mocks.profiles
import pymedphys._gamma.implementation.shell
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findbb

In [ ]:
image_path = pymedphys.data_path('wlutz_image.png')
image_path

In [ ]:
x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)
field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)

In [ ]:
edge_lengths = [20, 20]

field_centre, field_rotation = pymedphys._wlutz.findfield.find_centre_and_rotation(x, y, img, edge_lengths)
field_centre, field_rotation

In [ ]:
bb_diameter = 8
bb_radius = bb_diameter / 2
penumbra = 2

bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation)
bb_centre

In [ ]:
x_half_bound = edge_lengths[0]/2 + penumbra * 3
y_half_bound = edge_lengths[1]/2 + penumbra * 3

x_axis = np.linspace(-x_half_bound, x_half_bound, 200)
y_axis = np.linspace(-y_half_bound, y_half_bound, 200)

field_transform = pymedphys._wlutz.interppoints.translate_and_rotate_transform(field_centre, field_rotation)
bb_transform = pymedphys._wlutz.interppoints.translate_and_rotate_transform(bb_centre, 0)

x_field_interp, y_field_interp = pymedphys._wlutz.createaxis.transform_axis(x_axis, y_axis, field_transform)
x_bb_interp, y_bb_interp = pymedphys._wlutz.createaxis.transform_axis(x_axis, y_axis, bb_transform)

In [ ]:
def draw_by_diff(dx, dy, transform):
    draw_x = np.cumsum(dx)
    draw_y = np.cumsum(dy)

    draw_x, draw_y = pymedphys._wlutz.interppoints.apply_transform(draw_x, draw_y, transform)
    
    return draw_x, draw_y

In [ ]:
t = np.linspace(0, 2*np.pi)
circle_x_origin = bb_diameter/2 * np.sin(t)
circle_y_origin = bb_diameter/2 * np.cos(t)

In [ ]:
circle_x = circle_x_origin + bb_centre[0]
circle_y = circle_y_origin + bb_centre[1]

rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]

rect_crosshair_dx = [-edge_lengths[0]/2, edge_lengths[0], -edge_lengths[0], edge_lengths[0]]
rect_crosshair_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1]]

bb_crosshair = np.array([-bb_radius, bb_radius])

plt.figure(figsize=(10,10))
plt.contourf(x, y, img, 100)

plt.plot([bb_centre[0], ]*2, bb_crosshair + bb_centre[1], 'k', lw=1)
plt.plot(bb_crosshair + bb_centre[0], [bb_centre[1], ]*2, 'k', lw=1)

plt.plot(*draw_by_diff(rect_dx, rect_dy, field_transform), 'k', lw=3)
plt.plot(*draw_by_diff(rect_crosshair_dx, rect_crosshair_dy, field_transform), 'k', lw=1)

plt.plot([field_centre[0], bb_centre[0]], [field_centre[1], bb_centre[1]], c='C3', lw=3)

plt.plot(circle_x, circle_y, 'k', lw=3)

plt.plot(x_field_interp[0], x_field_interp[1], 'k', lw=0.5, alpha=0.3)
plt.plot(y_field_interp[0], y_field_interp[1], 'k', lw=0.5, alpha=0.3)

plt.plot(x_bb_interp[0], x_bb_interp[1], 'k', lw=0.5, alpha=0.3)
plt.plot(y_bb_interp[0], y_bb_interp[1], 'k', lw=0.5, alpha=0.3)

plt.axis('equal')
plt.xlim([-20, 20])
plt.ylim([-20, 20])

plt.colorbar()

In [ ]:
plt.plot(x_axis, field(*x_field_interp))
plt.plot(-x_axis, field(*x_field_interp))

In [ ]:
plt.plot(y_axis, field(*y_field_interp))
plt.plot(-y_axis, field(*y_field_interp))

In [ ]:
plt.plot(x_axis, field(*x_bb_interp))
plt.plot(-x_axis, field(*x_bb_interp))

In [ ]:
plt.plot(y_axis, field(*y_bb_interp))
plt.plot(-y_axis, field(*y_bb_interp))
# plt.plot([-bb_diameter/2, bb_diameter/2], [0.6, 0.6])